In [2]:
from langchain.document_loaders import TextLoader
# 定义文本加载器，指定读取的文件路径
loader = TextLoader("./documents/三国演义.txt")
documents=loader.load()
print(documents[0].metadata)


from langchain.text_splitter  import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100,chunk_overlap=20,    
                                      separators=[
                                                "\n\n",
                                                "\n",
                                                " ",
                                                ".",
                                                ",",
                                                "\u200b",  # Zero-width space
                                                "\uff0c",  # Fullwidth comma
                                                "\u3001",  # Ideographic comma
                                                "\uff0e",  # Fullwidth full stop
                                                "\u3002",  # Ideographic full stop
                                                "",
                                            ],)
split_documents=text_splitter.split_documents(documents)

{'source': './documents/三国演义.txt'}


In [3]:
# 加载embedding模型
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

encode_kwargs = {"normalize_embeddings": False}
model_kwargs = {"device": "cuda:0"}
embeddings_model= HuggingFaceEmbeddings(
    model_name='/mnt/wushaogui/huggingface/shibing624/text2vec-base-chinese/',
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

/tmp/ipykernel_14312/2788769724.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings_model= HuggingFaceEmbeddings(
/home/wushaogui/anaconda3/envs/py39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/wushaogui/anaconda3/envs/py39/lib/python3.9/site-packages/transformers/utils/generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register

In [7]:
from langchain.vectorstores import Chroma

persist_directory="VectorStore" # 数据库保存路径
db = Chroma.from_documents(split_documents, embeddings_model, persist_directory=persist_directory)
db.persist()

/tmp/ipykernel_14312/2049267115.py:5: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  db.persist()


# 相似度搜索

使用similarity_search 方法快速搜索向量数据库中的相似文本片段

In [6]:
query = "吕布上马"
docs = db.similarity_search(query)
print(len(docs),docs[0].page_content)

4 ，说吕布拱手来降，可乎？”卓大喜，观其人，乃虎贲中郎将李肃也。卓曰：“汝将何以说之？”肃曰：“某闻主公有名马一匹，号曰‘赤兔’，日行千里。须得此马，再用金珠，以利结其心；某更进说词：吕布必反丁原
